In [ ]:
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe
import configparser
import pandas as pd

## Read in your config and open spreadsheet

In [ ]:
def combine_sections(config, *sections):
    combined = {}
    for section in sections:
        combined.update(config[section])  # later sections override earlier ones
    return dict(combined)

# get config
cp = configparser.ConfigParser() 
cp.read('config.ini')
config = combine_sections(cp, "sheets", "clearances")

# Setup auth and client
SCOPES = [config.get('scopes'),]
creds = Credentials.from_service_account_file(config.get('credentials'), scopes=SCOPES)
client = gspread.authorize(creds)

# Open your spreadsheet by name or URL
spreadsheet = client.open(config.get('workbook'))

## Generate your list of approved volunteers

In [ ]:
data = spreadsheet.worksheet(config.get('sheet_clearances') ).get_values("A4:G")
# Convert to list of dicts
keys = data[0]
rows = data[1:]
cleared_volunteers = [row for row in data if row[3] == "TRUE"]
cleared_volunteers_df = pd.DataFrame( [dict(zip(keys, row)) for row in cleared_volunteers] )

not_cleared_volunteers = [row for row in data if row[3] != "TRUE"]
not_cleared_volunteers_df = pd.DataFrame( [dict(zip(keys, row)) for row in not_cleared_volunteers] )

cleared_volunteers_df["full_name"] = (
    cleared_volunteers_df["First Name"].str.strip().str.lower()
    + "|" +
    cleared_volunteers_df["Last Name"].str.strip().str.lower()
)

not_cleared_volunteers_df["full_name"] = (
    not_cleared_volunteers_df["First Name"].str.strip().str.lower()
    + "|" +
    not_cleared_volunteers_df["Last Name"].str.strip().str.lower()
)

## Read in the signup genius report

In [ ]:
def format_phone(num):
    if pd.isna(num) or num.strip() == "":
        return ""  # keep blanks as blanks
    num = "".join(filter(str.isdigit, num))  # strip non-digits just in case
    if len(num) == 10:  # US style number
        return f"{num[0:3]}.{num[3:6]}.{num[6:]}"
    return num  # fallback: leave as-is if not 10 digits

In [ ]:
sug_report = 'away_volunteers.csv'
df = pd.read_csv(
    sug_report,
    usecols=["Location", "First Name", "Last Name", "Email", "Phone", "Item", "Item Comment", "Sign Up", "Start Date/Time (mm/dd/yyyy)", "Sign Up Comment", "Sign Up Timestamp"],
    dtype=str
)
df["Phone"] = df["Phone"].apply(format_phone)

# get good start date/time columns
df["start_date_time"] = pd.to_datetime(
    df["Start Date/Time (mm/dd/yyyy)"],
    format="%m/%d/%Y %I:%M %p",
    errors="coerce"
)
df["start_date"] = df["start_date_time"].dt.date
df["start_time"] = df["start_date_time"].dt.time
df = df.drop(columns=["Start Date/Time (mm/dd/yyyy)", "start_date_time"])

# get rid of rows with no signups
df = df.dropna(subset=["Sign Up Timestamp"])

df["signup_timestamp"] = pd.to_datetime(
    df["Sign Up Timestamp"],
    format="%m/%d/%Y %I:%M:%S %p",
    errors="coerce"
)

# clean up email
df["Email"] = df["Email"].str.strip().str.lower()

# build a full name
df["full_name"] = (
    df["First Name"].fillna("").str.strip().str.lower()
    + "|" +
    df["Last Name"].fillna("").str.strip().str.lower()
)

volunteer_df = df
volunteer_df["Shift"] = volunteer_df["Item Comment"].str.extract(r"^(Shift [12])", expand=False)

## Build List of Cleared Volunteers

In [ ]:
# build a pattern for roles that don't need clearance
non_cleared_roles = [
    "Fry Trailer",
    "Booth Volunteer",
    "Kona Ice",
    "AGMB Spiritwear Sale"
]
non_cleared_roles_pat = "|".join(non_cleared_roles)

# Get a unique list of volunteers
unique_volunteers = volunteer_df[~volunteer_df["Item"].str.contains(non_cleared_roles_pat, na=False)][['First Name', 'Last Name', 'Email', 'Phone']].\
    sort_values(by=["Phone", "Email"], ascending=False).\
    drop_duplicates(subset=["First Name", "Last Name"], keep="first").\
    sort_values(by=["Last Name", "First Name"], ascending=True).\
    reset_index(drop=True)

# create helper columns to join on
unique_volunteers["full_name"] = (
    unique_volunteers["First Name"].str.strip().str.lower()
    + "|" +
    unique_volunteers["Last Name"].str.strip().str.lower()
)

# Create blank column
unique_volunteers["volunteer_cleared"] = "no-information"
unique_volunteers["match_method"] = "no-match"

# Check for not cleared by name
mask = (
    (unique_volunteers["match_method"] == "no-match")
    & (unique_volunteers["full_name"].isin(not_cleared_volunteers_df["full_name"]))
)
unique_volunteers.loc[mask, ["match_method", "volunteer_cleared"]] = ["name", "no"]

# Check for not cleared by Email
mask = (
    (unique_volunteers["match_method"] == "no-match")
    & (unique_volunteers["Email"].isin(not_cleared_volunteers_df["Email"]))
)
unique_volunteers.loc[mask, ["match_method", "volunteer_cleared"]] = ["email", "no"]

# Check for cleared by name
mask = (
    (unique_volunteers["match_method"] == "no-match")
    & (unique_volunteers["full_name"].isin(cleared_volunteers_df["full_name"]))
)
unique_volunteers.loc[mask, ["match_method", "volunteer_cleared"]] = ["name", "yes"]

# Check for cleared by Email
mask = (
    (unique_volunteers["match_method"] == "no-match")
    & (unique_volunteers["full_name"].isin(cleared_volunteers_df["Email"]))
)
unique_volunteers.loc[mask, ["match_method", "volunteer_cleared"]] = ["email", "yes"]

# cleanup helper column if you don’t need it
#unique_volunteers = unique_volunteers.drop(columns=["full_name"])
# save to google docs
sheet_status_list = "volunteer_list_w_status"
try:
    sheet_status = spreadsheet.worksheet(sheet_status_list)
except:
    sheet_status = spreadsheet.add_worksheet(title=sheet_status_list, rows=100, cols=20)

#warwick.batch_clear(["warwick"])
sheet_status.batch_clear(["A:M"])

#warwick.clear()
save_df = unique_volunteers.drop(columns=["full_name"])
set_with_dataframe(sheet_status, save_df.sort_values(by=["Last Name", "First Name"], ascending=True).reset_index(drop=True), row=1, col=1)
sheet_status.resize(rows=len(save_df)+1, cols=len(save_df.columns))


## Add in the clearance information and save to google sheets

### Add has_clearances column to the volunteer_df

In [ ]:
# mark as N/A if the role doesn't require clearances
volunteer_df["has_clearances"] = volunteer_df["Item"].str.contains(non_cleared_roles_pat, na=False).map(
    {True: "N/A", False: None}
)

# give me only my cleared volunteers
cleared = unique_volunteers[unique_volunteers["volunteer_cleared"].str.lower() == "yes"].copy()

# generate cleared emails
cleared_emails = set(cleared["Email"].dropna().str.strip().str.lower())

# generate cleared list of full_names
cleared["full_name"] = (
    cleared["First Name"].fillna("").str.strip().str.lower()
    + "|" +
    cleared["Last Name"].fillna("").str.strip().str.lower()
)
cleared_names = set(cleared["full_name"])

# Check on name first to see if cleared
volunteer_df.loc[
    volunteer_df["has_clearances"].isna() & volunteer_df["full_name"].isin(cleared_names),
    "has_clearances"
] = "yes"

# check on email next to see if cleared
volunteer_df.loc[
    volunteer_df["has_clearances"].isna() & volunteer_df["Email"].isin(cleared_emails),
    "has_clearances"
] = "yes"

# all other empty is not cleared
volunteer_df["has_clearances"] = volunteer_df["has_clearances"].fillna("no")

### Save to google docs

In [ ]:
sheet = "volunteer_roles"
try:
    sheet_status = spreadsheet.worksheet(sheet)
except:
    sheet_status = spreadsheet.add_worksheet(title=sheet, rows=100, cols=20)

sheet_status.batch_clear(["A:M"])
col_order = [
    "start_time",
    "start_date",
    "has_clearances",
    "First Name",
    "Last Name",
    "Item",
    "Shift",
    "Email",
    "Phone",

]
save_df = volunteer_df.drop(columns=["full_name", "Location", "Sign Up", "signup_timestamp", "Item Comment"])[col_order]
set_with_dataframe(sheet_status, save_df.sort_values(by=["start_date","Last Name", "First Name", "Shift", "Item"], ascending=True).reset_index(drop=True), row=1, col=1)
sheet_status.resize(rows=len(save_df)+1, cols=len(save_df.columns))